In [57]:
%matplotlib inline
#load libraries
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier #baseline model
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import Imputer

from IPython.display import display #displays full dataframe columns
#display all dataframe columns when printed
pd.options.display.max_columns = None

In [48]:
#load data
df = pd.read_csv('C:/Users/Mark.Burghart/Documents/projects/hospice_carepoint/data/transformed/carepoint_transformed_dummied.csv', index_col=0)
df.shape

(271541, 120)

Need to still create building/holdout sets (80/20), cross validation of building set, impute missing values (set to 0 for now).
<br>
<br>
Pipeline todo: missing data, feature selection, grid search hyperparameter tuning, model training.
<br><br>
pipe flow: Fill in missing data, feature selection, model training and tuning via grid search. 



### Train/Test Split


In [49]:
#separate variables (X) from outcome of interest (y)
df.shape
cols = df.columns.get_values() #converts column names to list
cols = cols.tolist()
feature_cols = [x for x in cols if x != 'death_within_7_days'] #removes outcome of interest from list ('death_within_7_days')

#extract rows
#print(feature_cols) #debug
X = df.loc[:, feature_cols]
X.shape #outcome column has been removed

(271541, 119)

In [50]:
#save outcome variable as y
y = df.death_within_7_days
y.shape

(271541,)

In [51]:
#separate data into training/test (aka holdout) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 23) #random_state for reproducibility (if needed)
#X_test, y_test should not be used until NO MORE decisions are being made. This is the final, FINAL validation, and more often just used for model performance and generalizability!

<br> 
# Pipepline for imputation, feature selection, and model training/tuning<br><br>
#### Logistic Regression with L1 Lasso Penalty, imputation = Median columnar value


First up: Logistic Regression models. Going to try both L1 and L2 Regularization.<br><br>
Note: For *MISSING DATA*,  I am **IMPUTING THE MEDIAN COLUMN VALUE**, as opposed to setting to 0, or kNN imputation as of now.

In [60]:
kbest = SelectKBest(f_classif) #select best 'k' features
logreg = LogisticRegression(penalty = 'l1', random_state = 0) #L1 Regularization
impute = Imputer(missing_values = 'NaN', strategy = 'median', axis = 0) #impute missing values: replacing NaNs with Median Column value for each column

#Pipeline for Logistic Regression with L1 Lasso Regularization
pipe_lr_l1 = Pipeline([('imputer', impute),
                       ('kbest', kbest),
                      ('lr', logreg)])

In [61]:
#parameters for grid search cross validation 
parameters = {'kbest__k': [40, 60, 80, 100], #building models with 40, 60, 80, and 100 most significant variables
                 'lr__C' : [0.01, 0.1, 1, 10, 100]} #tuning C for logistic regression

In [62]:
#grid search
grid = GridSearchCV(pipe_lr_l1, parameters, cv = 5, scoring = 'roc_auc') #run grid on parameters, 5-fold cross validation, AUC is evaluation metric

In [63]:
%%time
#fit models
grid.fit(X_train,y_train)


C:\tooling\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [86] are constant.
  UserWarning)
C:\tooling\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
C:\tooling\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [86] are constant.
  UserWarning)
C:\tooling\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
C:\tooling\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [86] are constant.
  UserWarning)
C:\tooling\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
C:\tooling\Anaconda3\lib\site-packages\sklearn\fe

Best estimator:
Pipeline(memory=None,
     steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('kbest', SelectKBest(k=100, score_func=<function f_classif at 0x00000238F1E4FAE8>)), ('lr', LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])


KeyError: 'logisticregression'

Model interpretations:<br><br>
feature [86] is constant, meaning the value is constant for all visits and patients (and should be removed from model).<br><br>
true_divide warning is likely due to feature[86], and is trying to divide by 0 variance...



In [66]:
print("Best estimator:\n{}".format(grid.best_estimator_)) #prints best model and pipeline
print("Logistic Regression step:\n{}".format(grid.best_estimator_.named_steps["lr"])) #prints logistic regression step of pipeline
print("Logistic Regression coefficients:\n{}".format(grid.best_estimator_.named_steps["lr"].coef_)) #prints coefficients of best estimator
print("Best Parameters: {}".format(grid.best_params_)) #outputs best parameters settings
print("Best cross validation score: {:.2f}".format(grid.best_score_)) #best produced cross validation score

Best estimator:
Pipeline(memory=None,
     steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('kbest', SelectKBest(k=100, score_func=<function f_classif at 0x00000238F1E4FAE8>)), ('lr', LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])
Logistic Regression step:
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Logistic Regression coefficients:
[[-0.14365795  0.02485366 -0.15901603  0.11801389  0.17137557 -0.05839532
  -0.00664398  0.08395521  0.05447684  0.09313099 -0.00438136  0.11874033
   0.1419672   0.11773742 -0.0911

In [82]:
#AUC scores for each validation set run
print("Grid scores on Development set: ")
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, grid.cv_results_['params']): #note imbedded grid_l2 will need to be adjusted to correct object
    print("%0.3f (+/-%0.3f) for %r"
         % (mean, std *2, params))

Grid scores on Development set: 
0.807 (+/-0.004) for {'kbest__k': 40, 'lr__C': 0.01}
0.807 (+/-0.004) for {'kbest__k': 40, 'lr__C': 0.1}
0.807 (+/-0.004) for {'kbest__k': 40, 'lr__C': 1}
0.807 (+/-0.004) for {'kbest__k': 40, 'lr__C': 10}
0.807 (+/-0.004) for {'kbest__k': 40, 'lr__C': 100}
0.813 (+/-0.003) for {'kbest__k': 60, 'lr__C': 0.01}
0.814 (+/-0.003) for {'kbest__k': 60, 'lr__C': 0.1}
0.814 (+/-0.003) for {'kbest__k': 60, 'lr__C': 1}
0.814 (+/-0.003) for {'kbest__k': 60, 'lr__C': 10}
0.814 (+/-0.003) for {'kbest__k': 60, 'lr__C': 100}
0.815 (+/-0.003) for {'kbest__k': 80, 'lr__C': 0.01}
0.816 (+/-0.003) for {'kbest__k': 80, 'lr__C': 0.1}
0.816 (+/-0.003) for {'kbest__k': 80, 'lr__C': 1}
0.816 (+/-0.003) for {'kbest__k': 80, 'lr__C': 10}
0.816 (+/-0.003) for {'kbest__k': 80, 'lr__C': 100}
0.815 (+/-0.003) for {'kbest__k': 100, 'lr__C': 0.01}
0.816 (+/-0.003) for {'kbest__k': 100, 'lr__C': 0.1}
0.817 (+/-0.002) for {'kbest__k': 100, 'lr__C': 1}
0.817 (+/-0.002) for {'kbest__k': 1

**Logistic Regression with L1 penalty and median missing data imputation** <br><br>
- Best model selected used parameters K = 100 (100 variables included), and C = 1 for LogReg model. 
- AUC scores ranged from 0.807 (40 variables, C = 0.01-100) to 0.817

#### Logistic Regression with L2 Ridge Penalty, imputation = Median columnar value

In [68]:
#Logistic regression with L2 Regularization penalty
kbest = SelectKBest(f_classif) #select best 'k' features
logreg_l2 = LogisticRegression(penalty = 'l2', random_state = 0) #L2 Ridge Regularization
impute = Imputer(missing_values = 'NaN', strategy = 'median', axis = 0) #impute missing values: replacing NaNs with Median Column value for each column

#Pipeline for Logistic Regression with L2 'Ridge' Regularization
pipe_lr_l2 = Pipeline([('imputer', impute),
                       ('kbest', kbest),
                      ('lr', logreg_l2)])
#parameters for grid search cross validation 
parameters = {'kbest__k': [40, 60, 80, 100], #building models with 40, 60, 80, and 100 most significant variables
                 'lr__C' : [0.01, 0.1, 1, 10, 100]} #tuning C for logistic regression

#grid search
grid_l2 = GridSearchCV(pipe_lr_l2, parameters, cv = 5, scoring = 'roc_auc') #run grid on parameters, 5-fold cross validation, AUC is evaluation metric



In [69]:
%%time
#fit models
grid_l2.fit(X_train,y_train)

C:\tooling\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [86] are constant.
  UserWarning)
C:\tooling\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
C:\tooling\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [86] are constant.
  UserWarning)
C:\tooling\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
C:\tooling\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [86] are constant.
  UserWarning)
C:\tooling\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
C:\tooling\Anaconda3\lib\site-packages\sklearn\fe

Wall time: 36min 16s


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('kbest', SelectKBest(k=10, score_func=<function f_classif at 0x00000238F1E4FAE8>)), ('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'kbest__k': [40, 60, 80, 100], 'lr__C': [0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [70]:
print("Best estimator:\n{}".format(grid_l2.best_estimator_)) #prints best model and pipeline
print("Logistic Regression step:\n{}".format(grid_l2.best_estimator_.named_steps["lr"])) #prints logistic regression step of pipeline
print("Logistic Regression coefficients:\n{}".format(grid_l2.best_estimator_.named_steps["lr"].coef_)) #prints coefficients of best estimator
print("Best Parameters: {}".format(grid_l2.best_params_)) #outputs best parameters settings
print("Best cross validation score: {:.2f}".format(grid_l2.best_score_)) #best produced cross validation score

Best estimator:
Pipeline(memory=None,
     steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('kbest', SelectKBest(k=100, score_func=<function f_classif at 0x00000238F1E4FAE8>)), ('lr', LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])
Logistic Regression step:
LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Logistic Regression coefficients:
[[-0.14429463  0.0249696  -0.15859719  0.11811835  0.17114564 -0.05717367
  -0.00658466  0.08416823  0.05471053  0.0933325  -0.0043772   0.11885902
   0.142829    0.1194861  -0.08

In [76]:
#AUC scores for each validation set run

print("Grid scores on Development set: ")
means = grid_l2.cv_results_['mean_test_score']
stds = grid_l2.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, grid_l2.cv_results_['params']): #note imbedded grid_l2 will need to be adjusted to correct object
    print("%0.3f (+/-%0.3f) for %r"
         % (mean, std *2, params))

Grid scores on Development set: 
0.807 (+/-0.004) for {'kbest__k': 40, 'lr__C': 0.01}
0.807 (+/-0.004) for {'kbest__k': 40, 'lr__C': 0.1}
0.807 (+/-0.004) for {'kbest__k': 40, 'lr__C': 1}
0.807 (+/-0.004) for {'kbest__k': 40, 'lr__C': 10}
0.807 (+/-0.004) for {'kbest__k': 40, 'lr__C': 100}
0.813 (+/-0.003) for {'kbest__k': 60, 'lr__C': 0.01}
0.814 (+/-0.003) for {'kbest__k': 60, 'lr__C': 0.1}
0.814 (+/-0.003) for {'kbest__k': 60, 'lr__C': 1}
0.814 (+/-0.003) for {'kbest__k': 60, 'lr__C': 10}
0.814 (+/-0.003) for {'kbest__k': 60, 'lr__C': 100}
0.815 (+/-0.003) for {'kbest__k': 80, 'lr__C': 0.01}
0.816 (+/-0.003) for {'kbest__k': 80, 'lr__C': 0.1}
0.816 (+/-0.003) for {'kbest__k': 80, 'lr__C': 1}
0.816 (+/-0.003) for {'kbest__k': 80, 'lr__C': 10}
0.816 (+/-0.003) for {'kbest__k': 80, 'lr__C': 100}
0.816 (+/-0.003) for {'kbest__k': 100, 'lr__C': 0.01}
0.817 (+/-0.003) for {'kbest__k': 100, 'lr__C': 0.1}
0.817 (+/-0.002) for {'kbest__k': 100, 'lr__C': 1}
0.817 (+/-0.002) for {'kbest__k': 1

<Br>**Logistic Regression with L2 penalty and median missing data imputation**<br><br>
- Best model selected used parameters K = 100 (100 variables included), and C = 10 for LogReg model. <br>
- AUC scores ranged from 0.807 (40 variables, C = 0.01-100) to 0.817 
- Interestingly, these outputs produced the same AUC across all cross validation trials as L1 model.
    -Only difference was C = 1 for L1. 
    - Could be due to same random_state parameter?<br>
    <br>
    
    Also interesting to note that adding 60 additional variables only improves AUC by ~1%. 100 variables is likely overfitting...
